## Load libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [70]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import pickle
import json
import matplotlib.pyplot as plt
from metrics import exact_match_metric
from callbacks import NValidationSetsCallback, GradientLogger
from generator import DataGenerator
import time

import sys
sys.path.append('../../')
from src.models.utils import concatenate_texts

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
print(tf.__version__)
print("GPU Available: ", tf.test.is_gpu_available())

2.0.0-beta1
GPU Available:  False


## Load settings

In [3]:
settings_path = Path('../../settings/settings.json')

In [4]:
with open(str(settings_path), 'r') as file:
    settings_dict = json.load(file)

In [5]:
settings_dict

{'math_module': 'arithmetic__add_sub',
 'train_level': 'easy',
 'batch_size': 1024,
 'thinking_steps': 0,
 'epochs': 1,
 'latent_dim': 256,
 'save_path': '/artifacts/',
 'data_path': '../../data/raw/v1.0/'}

## Load data

Start with batching a single file before tackling the whole dataset.

In [6]:
raw_path = Path(settings_dict['data_path'])
!ls {raw_path}

extrapolate  interpolate  train-easy   train-hard   train-medium


In [7]:
interpolate_path = raw_path/'interpolate'
!ls {interpolate_path} | head -5

algebra__linear_1d.txt
algebra__linear_1d_composed.txt
algebra__linear_2d.txt
algebra__linear_2d_composed.txt
algebra__polynomial_roots.txt


In [8]:
extrapolate_path = raw_path/'extrapolate'
!ls {extrapolate_path} | head -5

algebra__polynomial_roots_big.txt
arithmetic__add_or_sub_big.txt
arithmetic__add_sub_multiple_longer.txt
arithmetic__div_big.txt
arithmetic__mixed_longer.txt


In [9]:
train_easy_path = raw_path/'train-easy/'
!ls {train_easy_path} | head -5

algebra__linear_1d.txt
algebra__linear_1d_composed.txt
algebra__linear_2d.txt
algebra__linear_2d_composed.txt
algebra__polynomial_roots.txt


### Data settings

In [10]:
math_module = settings_dict["math_module"]
train_level = settings_dict["train_level"]

In [11]:
datasets = {
    'train':(raw_path, 'train-' + train_level + '/' + math_module),
    'interpolate':(interpolate_path, math_module),
    'extrapolate':(extrapolate_path, math_module)
           }

In [67]:
def concatenate_texts(path, pattern):
    file_paths = list(path.glob('{}*.txt'.format(pattern)))
    
    input_texts = []
    target_texts = []

    for file_path in file_paths:
        with open(str(file_path), 'r', encoding='utf-8') as f:
            lines = f.read().split('\n')[:-1]

        input_texts.extend(['\t' + input_text + '\n' for input_text in lines[0::2]])
        target_texts.extend(['\t' + target_text + '\n' for target_text in lines[1::2]])
        
    return input_texts, target_texts

In [71]:
%%time

input_texts = {}
target_texts = {}

for k, v in datasets.items():
    input_texts[k], target_texts[k] = concatenate_texts(v[0], v[1])
    print('Length of set {} is {}'.format(k, len(input_texts[k])))

Length of set train is 666666
Length of set interpolate is 10000
Length of set extrapolate is 10000
CPU times: user 271 ms, sys: 79.1 ms, total: 350 ms
Wall time: 350 ms


**Sample:**

In [72]:
random_idx = np.random.randint(1, len(input_texts['train']))
print('INPUT:', input_texts['train'][random_idx])
print('OUTPUT:', target_texts['train'][random_idx].strip())

INPUT: What is -2 - (1 - (3 + (5 - 3)))?
OUTPUT: 2


Concatenate texts to get text metrics (max length, number of unique tokens, etc.):

In [73]:
all_input_texts = sum(input_texts.values(), [])
all_target_texts = sum(target_texts.values(), [])

In [74]:
input_characters = set(''.join(all_input_texts))
target_characters = set(''.join(all_target_texts))

In [75]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in all_input_texts])
max_decoder_seq_length = max([len(txt) for txt in all_target_texts])

print('Number of samples:', len(all_input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 686666
Number of unique input tokens: 32
Number of unique output tokens: 13
Max sequence length for inputs: 99
Max sequence length for outputs: 6


In [76]:
all_input_texts[-1]

'Evaluate 37 + -41 - (267 + 0 + 0 + 5 + -20 + (46 - 35) - (-1 + 8)).'

In [77]:
all_target_texts[-1]

'\t-260\n'

## Process text

### Vectorise the text
Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping question characters to numbers, and another for answer characters to number.

In [50]:
# Creating a mapping from unique characters to indices
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [78]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [79]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='', char_level=True)
    lang_tokenizer.fit_on_texts(lang)
    
    tensor = lang_tokenizer.texts_to_sequences(lang)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
    
    return tensor, lang_tokenizer

In [80]:
input_tensor, inp_lang_tokenizer = tokenize(all_input_texts)
target_tensor, targ_lang_tokenizer = tokenize(all_target_texts)

In [81]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [82]:
max_length_targ, max_length_inp

(6, 99)

In [83]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

549332 549332 137334 137334


In [84]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [85]:
print ("Input Language; index to word mapping")
convert(inp_lang_tokenizer, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang_tokenizer, target_tensor_train[0])

Input Language; index to word mapping
16 ----> 0
1 ---->  
3 ----> +
1 ---->  
6 ----> (
4 ----> 1
4 ----> 1
1 ---->  
2 ----> -
1 ---->  
15 ----> 4
7 ----> )
1 ---->  
3 ----> +
1 ---->  
2 ----> -
26 ----> 7
1 ---->  
3 ----> +
1 ---->  
12 ----> 3

Target Language; index to word mapping
1 ----> 	
6 ----> 3
2 ----> 



In [86]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang_tokenizer.word_index)+1
vocab_tar_size = len(targ_lang_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [87]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 99]), TensorShape([64, 6]))

In [88]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [89]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 99, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [90]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [91]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 99, 1)


In [92]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [93]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 14)


In [94]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [96]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['\t']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [97]:
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))

        print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
        print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.0914
Epoch 1 Loss 0.0001
Time taken for 1 epoch 19.425013065338135 sec

Epoch 1 Loss 0.0003
Time taken for 1 epoch 21.822988033294678 sec

Epoch 1 Loss 0.0004
Time taken for 1 epoch 24.384517192840576 sec

Epoch 1 Loss 0.0007
Time taken for 1 epoch 26.82024908065796 sec

Epoch 1 Loss 0.0007
Time taken for 1 epoch 29.324955224990845 sec

Epoch 1 Loss 0.0009
Time taken for 1 epoch 31.91022515296936 sec

Epoch 1 Loss 0.0010
Time taken for 1 epoch 34.61877512931824 sec

Epoch 1 Loss 0.0011
Time taken for 1 epoch 37.23060607910156 sec

Epoch 1 Loss 0.0012
Time taken for 1 epoch 39.89896106719971 sec

Epoch 1 Loss 0.0013
Time taken for 1 epoch 42.680989265441895 sec

Epoch 1 Loss 0.0014
Time taken for 1 epoch 45.590203046798706 sec

Epoch 1 Loss 0.0014
Time taken for 1 epoch 48.44242000579834 sec

Epoch 1 Loss 0.0015
Time taken for 1 epoch 51.15497016906738 sec



KeyboardInterrupt: 